# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
#last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 2兆美元經濟紓困方案將達成協議　國際油價續漲

▲國際油價續漲。（圖／達志影像／美聯社）
文／中央社紐約24日綜合外電報導
由於美國聯邦準備理事會（Fed）推出空前收購債券計畫，美國參眾兩院國會議員表示即將就2兆美元經濟紓困方案達成協議，國際油價今天續漲。
紐約商品交易所西德州中級原油5月交割價上揚65美分，收在每桶24.01美元。
倫敦北海布倫持原油5月交割價上揚12美分，以每桶27.15美元作收。
----------------------------------------------------------------------

[1] 道瓊狂飆2112點！創87年來最大漲幅　美股重回2萬點大關

▲美股24日單日漲幅創下1993年以來的最高紀錄。（圖／路透社）
記者張靖榕／綜合外電報導
美國財政部長梅努欽（Steven Mnuchin），以及參議院民主黨領袖舒默（Chuck Schumer）於23日晚間透露，2兆美元的巨型紓困刺激方案已經「極為接近」達成協議；美國聯準會（Fed）當天也宣布推出「無上限量化寬鬆」，雙雙刺激美股強力反彈，道瓊更瘋狂飆漲超過2000點，創下1933年以來的紀錄。
道瓊工業指數終場上漲2112.98點，或11.37％，收在20704.91點；標普500指數上漲209.93點，或9.38％，收在2447.33點；那斯達克指數上漲557.18點，或8.12％，終場以7417.86點作收。
美股過去近三周發生多次熔斷，川普政府先後祭出千億美元的紓困法案，Fed更最後更降息4碼至0.25％-0％，仍難擋道瓊下探18000點。梅努欽先前向市場釋出消息，原本總額8000億美元，最終大幅加碼到2兆美元的財政刺激方案，會於23日由參議院通過；然而因當中5000億美元「政府自由用款」引發民主、共和兩黨意見分歧，最終流標。
當天晚上，梅努欽再次釋出方案即將通過的消息，這次民主黨領袖舒默也共同發布好消息，表明24日極有可能通過方案。
民主、共和兩黨周二（24日）上午再次透露「方案接近完成」，激勵市場信心，道瓊開盤即猛漲千點，率領三大指數飆升超過8％，重新站上2萬點大關。
-------------

▲台股盤中逐筆交易上路，金管會呼籲，投資人應視本身風險承受能力及投資策略，選擇適合的委託單種類。（示意圖／記者湯興漢攝）
記者紀佳妘／台北報導
台股盤中逐筆交易上路首日傳出災情，不少人反應下單APP「瘋狂轉圈圈」，市場也擔憂新制實施後會嚇跑投資人。金管會證期局今（24）日表示，證交所或券商交易主機，系統一切正常，後續也一切順暢，另觀察投資人結構，上路前一天自然人交易比重為51.4%，而上路首日自然人比重提升至54.4%，顯示散戶交易量未因逐筆交易上路後就減少，是一個中性的制度。
昨天是逐筆交易新制實施的第一天，不少投資人傳出有當機的情形發生，金管會證期局副局長蔡麗玲表示，初步了解後，無論是證交所或券商交易主機，系統一切都正常，而投資人反應無法下單，主要是少數券商的看盤軟體，開盤前有許多使用者同一時間更新軟體，導致投資人不太順暢，不過一發現問題後，隨時排除問題。
蔡麗玲指出，台股成交量集中市場1,540.5億元、上櫃市場281.75億元，累計1,822.25億元，比近期日均量2,000億元上下，相對來得低，不過若與去年日均量1,556.36億元相比，相對比較高，推測交易量與近期成交量較少的原因是國際情勢變化大，以及最近採行穩定股市措施，包括調降借券賣出量、有條件禁止平盤下放空等，對成交量也會有所影響。
另觀察投資人結構，蔡麗玲說，自然人交易比重昨天來到54.4%，與前一個交易日51.4%相比，顯示未因逐筆交易而減少，顯示逐筆交易制度是一個「中性制度」，因此沒有逐筆交易上路後，影響量能的情況發生。
蔡麗玲呼籲，中長期布局的散戶，建議可使用限價單ROD的委託方式，風險相對比較可以控制，且逐筆交易上路首日，使用限價單ROD者高達96.31%，顯示大部份投資人都能理解其風險，這是一個好現象。
----------------------------------------------------------------------

[16] 台灣資金充裕！　央行：有足夠流動性支應金融情勢變動

▲ 中央銀行。（圖／資料照）
記者李瑞瑾／台北報導
央行今（24）日公布109年2月金融情況，2月日平均貨幣總計數M1B、M2月增率分別為0.82%及0.76%，均較上月為低。M1B及M2年增率分別下降為7%及4.35%，主要受外資轉呈淨匯出之影響。累計本年1至2月M1B及M2平均年

▲虛擬貨幣本月歷經2次血洗，上周開始出現反彈行情。（圖／路透）
記者王曉敏／綜合報導
儘管股市持續受到新冠肺炎疫情的壓制，但虛擬貨幣行情今（24）日仍持續上漲。截至下午4點30分左右，主要虛擬貨幣24小時內的漲幅都超過10%以上，據Coinmarketcap.com數據，虛擬貨幣總市值在24小時內已增長了238億美元，達1,867.3億美元。
據Coindesk報價，截至稍早，比特幣24小時內漲幅達15.65%至6,770.17美元；以太幣上漲15.16%至142.06美元；萊特幣與波場幣也也分別上漲10.15%及11.18%。
歐美股市周一（23日）再度下跌，但虛擬貨幣行情卻逆勢上漲。此前，虛擬貨幣本月歷經2次血洗，3月中旬比特幣更一度腰斬，價格跌破4,000美元大關，吐回今年以來所有漲幅，整個虛擬貨幣市場總值一日內蒸發935億美元。在長達2周的拋售潮後，虛擬貨幣上周開始出現反彈，不過目前比特幣的交易價格仍低於今年初水準。
Coindesk資深分析師Noelle Acheson撰文指出，對於那些擔心經濟急遽萎縮的人來說，「比特幣實際上是唯一不受宏觀經濟『直接』影響的資產。」沒有什麼可削減的收入，也沒有阻礙供應的供應鏈，能源成本及供應鏈之類的外部因素可能會影響比特幣礦工的經濟狀況，但比特幣本身會根據網路的變化進行調整。
CNBC報導，加密貨幣交易所Luno業務開發主管Vijay Ayyar表示，聯準會（Fed）23日宣布啟動「無限制量化寬鬆」政策救市，「我們看到比特幣的價格走勢及其它資產類別出現了真正的上漲。這給投資人很大的信心。」他指出，此為比特幣在低迷市場首次有機會展現其避險特性，「結果仍有待驗證，看看比特幣在這種環境下的表現如何。」
----------------------------------------------------------------------

[28] 股神慘賠！美股一個月跌38%　謝金河：巴菲特被套牢

● 謝金河／《財訊》社長兼發行人、財信傳媒董事長
這回連股神也賠慘了！每次在股炎來襲的時候，最喜歡告訴大家危機入市的股神巴菲特，在這一次美國股市11354點的瘋狂跌勢中，巴菲特逢低買進，也套牢不少股票。
▲ 柏克夏A股是這次美股重災區 。（圖／翻攝謝金河臉書）
像是美國四大航空公司股票，從美國航空、聯合航空、西南航空到達美航空

● 林建甫／中信金融管理學院講座教授、台大經濟系名譽教授
新冠疫情蔓延，全球股市的大跌，美股從2009年3月至今長達11年的牛市，遭到扭轉成為熊市，全球股市也從多頭市場轉為空頭市場。
聯準會下猛藥救市　恐生流動性危機
新冠疫情出現前，美聯準會的指標利率：聯邦資金率（Federal Fund rate）利率水準在1.5%至1.75%區間，遠低於2001年互聯網泡沫危機及2008年全球金融危機兩次危機前的水準，降息空間較小。
聯準會原本要在3月17、18日才召開例行性會議討論降息。但是等不及，3月3日的緊急會議趕緊降息兩碼，這已經超出每次一碼的慣常力度。之後突然又在3月15日召開緊急會議，趕在週一亞洲市場開市前發揮作用，利率直接下調4碼，降至0-0.25%的歷史低位。
美聯準會在前兩次危機中，第一次的2001年1月起連續降息13次，將利率從6.5%降至1.0%，而第二次則從5.25%降到0至0.25%的超低水準。
然3月15日的緊急降息，力度上一步到位，但金融市場並未給予積極回應，紐約股市16日開盤暴跌並再次觸發熔斷機制，重啟交易後跌幅擴大，收盤時三大股指跌幅均超過11%。美股道瓊指數還暴跌2997點，創史上最大單日跌點。之後迄今，金融市場仍然劇烈動盪，美股跌勢未能止住。
 ▲學者認為，美聯準會「恐慌救市」下猛藥，既無助控制疫情，反而引發投資者疑慮升溫。（圖／路透社）
聯準會大幅降息　疑具政治考量
為什麼美聯準會沒有算到這些，而仍執意的一次大幅降息？這有可能是聯準會失去了「獨立性」，主席鮑爾為了保官位的結果。
在3月5日聯準會的降息後，美國總統川普對聯準會主席鮑爾又表達了不滿意。14日川普更在疫情的說明會後，就訴說鮑爾的不是，也稱有權開除鮑爾。他威脅可能會把他降職（demote），重新找一位聯準會主席。川普一再痛批鮑爾做了很多糟糕的決定，聯準會也沒有做該做的事，不像歐洲各國央行行動的積極。
▲學者認為，聯準會大幅降息，可能是聯準會主席鮑爾為了保官位，迎合川普。（圖／路透社）
聯準會主席鮑爾上任背景
其實從競選開始，川普就把炮火對準了聯準會。川普在競選辯論時，就抨擊當時的美聯準會主席葉倫，對他的政策說三道四。
葉倫屆滿後，川把沒有任何經濟學背景的鮑爾，推上了新一屆聯儲主席的位置。律師出身的鮑爾，曾被嘲笑為「非典型」總統任命的「非典型」聯準會主席。
也許是因為缺乏經濟學背景

▲蘋果先前提出產品關稅豁免的請求，目前只有Apple Watch成功獲得完全減免。（圖／《THE VERGE》）
記者謝仁傑／綜合報導
去年9月，川普總統對從中國進口的一系列產品，包括Apple Watch在內，課徵15％的關稅，作為中美貿易戰的武器之一，不過，隨著中美雙方簽訂貿易協定，這項關稅也在今年2月削減了一半，降至7.5％。
如今，美國貿易代表署又同意讓Apple Watch豁免關稅，對於蘋果來說，無疑是一大好消息。
蘋果向美國貿易代表署的最初請求中表示，Apple Watch在戰略上不重要，也與其他中國工業計劃無關；蘋果還表示，它尚未在中國以外的地方，找到可以滿足美國對Apple Watch需求的貨源。
去年11月，蘋果向美國政府提出關稅豁免的請求，清單包括11種產品和組件，例如AirPods、HomePod、Beats系列產品、iPhone零組件等；蘋果也成功地獲得部分Mac Pro零組件的關稅減免，不過並非所有減免請求都獲得批准。
先前蘋果執行長庫克（Tim Cook）和川普總統會面時，分析了課徵關稅可能對蘋果業務造成的影響。去年8月，川普表示庫克提出的「關稅將影響蘋果與三星的競爭力」是一個具有說服力的論點。
----------------------------------------------------------------------

[52] 竹科第一家！台積電啟動分流辦公　部分員工已「在家上班」

▲台積電上周五部分員工已在家辦公。（圖／資料照）
記者周康玉／台北報導
新冠肺炎疫情日益嚴峻，晶圓代工龍頭廠台積電上周五向員工發「分流辦公措施」的內部信中，按分流方式及配套措施，已有部分員工在家上班，台積電現約有45000名員工皆在此名單中，是新竹科學園區當中首家在家辦公的科技大廠。
因台積電上周三一名員工確診，經過台積電嚴格追朔其接觸史，發現該員工有近距離接觸30人，除了將這30人進行隔離外，上週五進一步實施分流辦公，在此分流辦公方法中，也就是同單位的員工分成A、B兩組，並分區上班外，當中也包含居家辦公。　
台積電表示，分流辦公大方向分成A、B兩組，有些單位要再細分A1和A2、A3、A4組等，有的是在不同樓層，有的是在家，大原則是：非生產直接相關人員能在家就盡量在家，正式實施在家辦公。
台積電表示，分流辦公辦法會按照該組或員工的職務需求

▲曾介入美女醫師蔡佳芬婚姻的漢唐光電董事長徐國良，今遭判刑定讞。（圖／翻攝自Facebook／蔡佳芬醫師 生活·美學手札）
記者吳銘峯／台北報導
曾與知名皮膚科女醫師蔡佳芬爆出婚外情的漢唐光電董座徐國良，不倫事件最後更造成女醫師與丈夫離婚。此外徐國良被檢調單位查出，以不實財報、誆騙上市櫃為由吸金3億多元，提起公訴。一二審均將他判刑8年半，其中二審更查出，徐國良的公司根本沒從事科技業，僅經營連鎖咖啡店而已。案經上訴，最高法院24日駁回上訴定讞。
媒體在2016年間爆料，徐國良介入知名皮膚科女醫師蔡佳芬與台大醫院外科主治醫師陳克誠間的婚姻，還說蔡佳芬為了向徐國良索討2億元，而與徐國良發生關係懷孕，但最後卻墮胎。最後媒體消遣蔡佳芬錢沒拿到，幸福的家庭也已經破滅，3人間也爆發多起官司。
徐國良因為介入美女醫師的婚姻而出現在新聞版面上，另外檢調單位也查出，徐男從2004年起，就以「印股票、換鈔票」方式吸金，徐國良先虛增公司資本額為6億元，並對外謊稱漢唐光電營運良好，將於近年內上市櫃，或美國思科公司擬併購漢唐光電公司，併購金額極高，股價將大幅翻漲，若未上市櫃或併購未成功，還會全額退還股款，藉此誘騙投資人購買漢唐光電公司股票。
徐男還捏造自己是具有史丹佛大學博士學位及美國貝爾實驗室副總裁、在美獲獎無數的學經歷，取信被害人，被害人紛紛掏錢投資，總計吸金3億8,037萬餘元。一審法院將他判刑8年半，案經上訴，高院二審更查出，徐男的學經歷全都是假造，漢唐光電也從未經營科技業，實際營業項目為經營實體咖啡店「酷咖啡」及網路平台，主要營收來自咖啡店，但仍入不敷出，長年處於虧損狀態，營收狀況極差。公司也沒有廠房、員工又少，沒有能力從事光電業，也不可能被美國公司收購，一切都是謊言。高院因此仍將他判刑8年6月。
案經上訴第三審，最高法院審理後，24日駁回上訴，徐國良判刑8年半定讞。而徐國良早在偵查期間就被羈押，期間雖法院裁定將他3000萬元交保，但覓保無著，至今仍羈押在看守所中。
----------------------------------------------------------------------

[65] 亞股早盤齊漲！日股、韓股直線漲破6％、台股緊追在後

▲亞股今(23)早盤齊漲，各國皆以高於平盤開出，且一路上漲。（資料照／記者湯興漢攝）
記者謝仁傑／綜合報導
亞洲股

▲英國保誠人壽。（圖／英國保誠人壽提供）
記者楊絡懸／台北報導
英國保誠人壽23日宣布，即日起與王道銀行合作，用戶可以透過王道銀行App或網路銀行購買保險商品，包括選購、試算、投保與核保等功能，都可在網路上直接處理，一次滿足所有需求；而保險商品如意外險，也可因應需求分別購買，另有定期壽險、醫療險及旅平險供選擇，提供全方位保障。
英國保誠人壽總經理王慰慈表示，王道銀行是台灣第一家原生數位銀行，在數位金融服務上，累積了豐富的客戶經驗，也積極地拓展年輕市場，與王道銀行攜手共同推出網路投保活動，正是彰顯英國保誠運用科技以為客戶創造更佳服務的承諾。
▲英國保誠人壽總經理王慰慈。（圖／記者湯興漢攝）
王慰慈說，英國保誠人壽致力讓客戶更輕鬆、便利獲得保險保障，「我們了解客戶的需要，特別是身處數位時代的千禧一族，尋求簡單、迅速、且貼合日益繁忙生活的解決方案，輕鬆從網路上了解商品，享受無縫點到點的購買體驗；面對客戶在數位時代不斷變化的需求，我們的首要任務是力求創新，以幫助台灣民眾保障健康及增長財富。」
王道銀行表示，王道銀行致力提供便利、安全、貼心的數位金融服務，希望藉由數位金融低門檻、無疆界、零時差的特色，為消費者生活創造更多便利性。王道銀行很高興與英國保誠人壽合作，提供消費者更完整的網路投保服務，使王道銀行的產品線更多樣與完善。
▲王道銀行。（圖／《ETtoday新聞雲》資料照）
英國保誠人壽表示，網路投保平台提供「碎片化商品組合」，民眾可自行選擇符合需求的投保商品並且自由組合，再透過獨創的「保險購物車」功能，將符合需求的商品隨手放進購物車，簡單選購投保；防疫期間，減少出門的風險，可以透過網路投保購買保障。
此外，英國保誠人壽與王道銀行也推出「王道有禮」網路投保活動，即日起至5月31日，王道銀行客戶透過王道銀行App或網路銀行的「O速保專區」，連結至英國保誠人壽網路投保，並以王道銀行簽帳金融卡進行繳費，除了享王道銀行簽帳金融卡1%的現金回饋，還可額外再享3%的現金回饋，刷卡繳保費的現金回饋共計4%，回饋金額無上限。
----------------------------------------------------------------------

[80] 快訊／美股期指跳漲400點！WTI原油期激升4％…Fed宣布新救市計畫

▲美股上周跌幅達17％，創下2008年以

▲ 金管會顧立雄（右四）及證期局局長王詠心（左四），赴臺灣證券交易所視察撮合系統之運作情況，由證交所董事長許璋瑤（右三）及總經理簡立忠（右二）接待。（圖／證交所）
記者李瑞瑾／綜合報導
臺灣證券市場於今（23）日起實施盤中全面逐筆交易，證交所表示，實施首日證交所及證券商系統皆正常運作，經統計，總委託筆數計1,341萬6,649筆，其中限價委託占99.67%，市價委託占0.33%，顯示投資人仍以使用限價委託為主。
證交所指出，為關心新制實施首日交易系統運作情形，金管會主委顧立雄在證期局局長王詠心陪同下，赴證交所視察撮合系統之運作情況，由證交所董事長許璋瑤接待。證交所表示，實施首日交易系統均順利運行，配合逐筆交易之相關重大系統亦順利上線。
▲ 逐筆交易上路首日，證交所總經理簡立忠赴元大證券視察。（圖／證交所）
另外，證交所總經理簡立忠及櫃買中心總經理李愛玲率相關經理人親赴全台北中南等地共8家證券商營業處所（元大、富邦、凱基、群益金鼎、永豐金、大昌、光和及致和）瞭解新制上線市場運作情形，透過實地訪察，瞭解證券商從業人員接單及下單作業。
證交所指出，本次盤中逐筆交易新制順利施行，將持續關注整體市場是否有結構性變化，例如觀察大型股或中小型股的成交量及價格波動情形；分析整體市場投資人結構是否改變，並追蹤演算法交易或高頻交易的發展情形，必要時擬定相關規範予以控管，以維持市場結構的均衡，保障投資大眾權益。
----------------------------------------------------------------------

[91] 歐股下跌！　英、法、德股早盤重挫逾3％

▲歐洲股市本週開盤仍持續下墜。（圖／路透）
記者謝仁傑／綜合報導
新冠肺炎（COVID-19）疫情在歐美衝擊擴散，歐股上週五收盤也紛紛來到一個月以來的新低點。今（23）日早盤仍持續下墜，英、法、德股市目前皆重挫4％。
美股短短一個月自高點以來跌幅已3成，歐股也不遑多讓，英股一個月來下跌近3成，法股、德股皆下跌約3成5，西股下跌約3成3。
歐股今日表現（截至台灣時間下午6點07分）
英國FTSE100指數下跌229點，至4,961點，跌幅4.68％。
法國CAC40指數下跌129點，至3,919點，跌幅3.33％。
德國DAX指數下跌284點，至8,644點，跌幅3.33％。
西班牙IBE